In [2]:
import openai
from models.speech.model import transcribe_audio 
from models.diarisation.model import get_speaker_segments
from models.disclaimer.model import disclaimer_verifier
from models.disclaimer.constants import disclaimer_text
from models.sentiment.model import return_call_sentiment
from models.summariser.gpt_model import get_gpt_call_summary
from models.gpt_prompts import call_summary_instructions
from preprocessing.utils import convert_audio_to_wav, is_audio_mono, convert_stereo_audio_to_mono, load_text_file

### Audio Preprocessing

In [3]:
audio_path = "data/Call-1-Example.mp3"
clean_data_dir = "cleaned_data/"

Analysis will not work on MP3 files, so the audio data is converted to a WAV file.

In [4]:
new_audio_path = convert_audio_to_wav(audio_path, clean_data_dir)

Using stereo audio will result in failure during speaker diarization, so it needs to be converted to mono.

In [5]:
if not is_audio_mono(new_audio_path):
    new_audio_path = convert_stereo_audio_to_mono(new_audio_path, clean_data_dir)

### Speach Recognition

* Utilize OpenAI's transcription service to transcribe the audio.
* Employ Speechbrain's speaker diarization module to identify speakers.

The num_speakers argument is required for speaker diarization.

In [6]:
results = transcribe_audio(new_audio_path, num_speakers=2)

### Speaker Diarization 

It is now possible to organize speech transcription based on the speaker's identity.

In [7]:
speaker_segments_dict = get_speaker_segments(results["segments"])

In [8]:
speaker_segments_dict["SPEAKER 2"][0]

{'text': "Hi Anna, my name is Bob.  I'm calling because I've been tracking a parcel that was supposed to be delivered to me three days ago.  But the status hasn't updated since it was out of delivery.  Can you help me with that?",
 'start': 25.52,
 'end': 40.04,
 'segment_num': 4}

In [9]:
speaker_segments_dict["SPEAKER 1"][0]

{'text': 'Good morning, thank you for calling the Ghost Office, the most-isperated postal service.  My name is Anna, how can I assist you today?  Before we proceed, I need to inform you that this call is being recorded.  We may contact you in the future to offer future products and services.  You can always have the option to withdraw from receiving this contact from us.  Now, how can I help you today?',
 'start': 0.0,
 'end': 25.04,
 'segment_num': 6}

### Disclaimer Verification

To determine which segment to compare with the disclaimer text, I executed a for loop on both speakers' segments and computed similarity scores for each segment. Then, only the maximum score is returned.

Disclaimer verification rules:
1. If the similarity score is less than 50%, 'false' will be returned.
2. If the speaker utters the disclaimer after 45 seconds, 'false' will be returned.
3. If the speaker utters the disclaimer after reaching the third segment in the speaker_segments_dict, 'false' will be returned.


In this example, I am comparing the following two passages of text:

In [10]:
disclaimer_text

'I need to inform you that this call is recorded. We may contact you in the future to offer further products and services. You always have the option to withdraw from receiving this contact from us'

In [11]:
speaker_segments_dict["SPEAKER 1"][0]["text"]

'Good morning, thank you for calling the Ghost Office, the most-isperated postal service.  My name is Anna, how can I assist you today?  Before we proceed, I need to inform you that this call is being recorded.  We may contact you in the future to offer future products and services.  You can always have the option to withdraw from receiving this contact from us.  Now, how can I help you today?'

In [12]:
disclaimer_verifier(speaker_segments_dict)

True

### Sentiment Analysis



A sentiment analysis model is utilized to determine the overall tone of the conversation. A custom neutral threshold is employed to specify the strictness of this model.

In [13]:
return_call_sentiment(results["text"])

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


'POSITIVE'

### Call Text Summary

GPT 3.5 test example. I copied and pasted the full prompt into ChatGPT. 

The full prompt will include a list of instructions and a string version of the speaker segment dict.


A list of of instructions is given to the gpt model before the output

GPT Instructions:

* "You will be asked to summarised information about a phone call."
* " Make sure to include bullet points on what each person said and bullet points on action items"

Rules to follow:
* " concise and informative"
* " capturing key points and action items." 
* " a format that is easily understandable and highlights important aspects of the conversation."



In [14]:
client = openai.OpenAI(api_key=load_text_file("openai_key.txt"))

In [15]:
response = get_gpt_call_summary(client, str(speaker_segments_dict))

In [16]:
print(response.choices[0].message.content)

**Summary:**
- **Speaker 1: Anna**
    - "Good morning, thank you for calling the Ghost Office. How can I assist you today?"
    - Requested Bob for tracking number to resolve parcel issue.
    - Informed Bob about delay in delivery due to unexpected reroute but assured delivery by tomorrow.
    - Offered to sign Bob up for SMS notifications for updates on parcel's journey and delivery time window.
    - Concluded call by offering further assistance and looking forward to parcel's delivery tomorrow.

- **Speaker 2: Bob**
    - Bob called to inquire about a parcel that was supposed to be delivered three days ago but with no status update since out for delivery.
    - Provided tracking number GH123456789.
    - Expressed relief upon hearing about the delay reason.
    - Opted for SMS notifications for updates to avoid missing delivery.
    - Expressed gratitude for Anna's assistance and confirmed no further assistance needed.
    - Thanked Anna and expressed well wishes.

**Action Items: